## Collecting UN Global Compact Communication of Progress (COP) Annual Reports

Web scrapping from [here](https://www.unglobalcompact.org/participation/report/cop/create-and-submit/active) for all COP reports submitted up to 2021. We only take into account the reports submitted in English.

Last updated on August 19, 2021

Please specify below the focus year of this analysis:

In [3]:
focus_year = "2021"

Please select the focus language using one of the following values:
- en (English, default option)
- de (German)
- es (Spanish)
- fr (French)
- pt (Portuguese)

In [4]:
focus_language = 'en'

In [5]:
language_ref = { 'en' : { 'name' : 'English', 'min_coocurrence' : 10},
                 'de' : { 'name' : 'German', 'min_coocurrence' : 2},
                 'es' : { 'name' : 'Spanish', 'min_coocurrence' : 2},
                 'fr' : { 'name' : 'French', 'min_coocurrence' : 2},
                 'pt' : { 'name' : 'Portuguese', 'min_coocurrence' : 2},
               }

## 1. Gather document index information about COP reports available from the website
The [UN Global Compact website](https://www.unglobalcompact.org/participation/report/cop/create-and-submit/active) contains entries for each COP report, describing the sector of the company submitting the report, country and year, as well as the language in which the repoort was written in and a link to a PDF file with the full report.

**The results in this section give a general view of the available COPs, it's not yet restricted by the focus_year and focus_language.**

In [6]:
!pip install --upgrade pip
!python -m pip install PyPDF2
!pip install --upgrade setuptools
!pip install requests
!pip install stop-words

     |████████████████████████████████| 1.6 MB 1.3 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.1
    Uninstalling pip-21.2.1:
      Successfully uninstalled pip-21.2.1


In [7]:
import requests
import re
import pandas as pd
import PyPDF2
import shutil
import nltk
import os
import os.path
import requests
from urllib.request import urlretrieve
from bs4 import BeautifulSoup

In [8]:
#Display the total number of CoPs available

gc_url = "https://www.unglobalcompact.org/participation/report/cop/create-and-submit/active?page=1&per_page=250"
gc_base_url = "https://www.unglobalcompact.org"
gc_home = requests.get(gc_url)

soup = BeautifulSoup(gc_home.content, 'lxml')
header = soup.h2.string

total_num_cops = re.search(r'(?<=: )[0-9]+', header)[0]
print("Total number of COPs available: %s" % total_num_cops)

Total number of COPs available: 55060


In [9]:
#Create a BeautifulSoup to parse all available HTML page
full_gc_url_part1 = "https://www.unglobalcompact.org/participation/report/cop/create-and-submit/active?page="
full_gc_url_part2 = "&per_page=250"

def get_link(page):
    
    r = requests.get(full_gc_url_part1+str(page)+full_gc_url_part2)
    soup = BeautifulSoup(r.content, 'lxml')
    #links = [td.find_all('a')[0]['href'] for td in soup.find_all('td', 'participant')]

    return soup

#print("Getting full list of reports ...")
gc_full_list_soup=BeautifulSoup()
for i in range(0,219): #Current max page; Need to implement len(max page)
  gc_full_list_soup.append(get_link(i))

#gc_full_list_soup = BeautifulSoup(set_of_links.content, 'lxml')

/Users/ohikendoit/.local/lib/python3.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/ohikendoit/opt/anaconda3/lib/python3.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [ ]:
#check the existing 'th' class from the soup object
#removing of 'th' is required to collect only the gc_full_list_soup.find_all("tr") cell
element = gc_full_list_soup.find_all('th')

#remove th class
for th in gc_full_list_soup('th'):
  th.decompose()

#remove tags that have no content
for x in gc_full_list_soup.find_all():
  if len(x.get_text(strip=True)) ==0:
    x.extract()

#check the results
gc_full_list_soup

In [ ]:
#Check SDGs contribution from each company
def check_sdgs(profile):
    has_sdg1 = "no"
    has_sdg2 = "no"
    has_sdg3 = "no"
    has_sdg4 = "no"
    has_sdg5 = "no"
    has_sdg6 = "no"
    has_sdg7 = "no"
    has_sdg8 = "no"
    has_sdg9 = "no"
    has_sdg10 = "no"
    has_sdg11 = "no"
    has_sdg12 = "no"
    has_sdg13 = "no"
    has_sdg14 = "no"
    has_sdg15 = "no"
    has_sdg16 = "no"
    has_sdg17 = "no"

    questions = profile.find_all("ul", class_='questionnaire')
    if len(questions) == 2:
        sdgs = questions[0].find_all("li")
        if len(sdgs) != 18:  # the correct SDG questionnaire has 17 questions + header
            temp_sdgs = questions[1].find_all("li")
            if len(temp_sdgs) == 18:
                sdgs = temp_sdgs
            else:
                sdgs = []
        if 'selected_question' in sdgs[1].get('class'):
            has_sdg1 = "yes"
        if 'selected_question' in sdgs[2].get('class'):
            has_sdg2 = "yes"
        if 'selected_question' in sdgs[3].get('class'):
            has_sdg3 = "yes"
        if 'selected_question' in sdgs[4].get('class'):
            has_sdg4 = "yes"
        if 'selected_question' in sdgs[5].get('class'):
            has_sdg5 = "yes"
        if 'selected_question' in sdgs[6].get('class'):
            has_sdg6 = "yes"
        if 'selected_question' in sdgs[7].get('class'):
            has_sdg7 = "yes"
        if 'selected_question' in sdgs[8].get('class'):
            has_sdg8 = "yes"
        if 'selected_question' in sdgs[9].get('class'):
            has_sdg9 = "yes"
        if 'selected_question' in sdgs[10].get('class'):
            has_sdg10 = "yes"
        if 'selected_question' in sdgs[11].get('class'):
            has_sdg11 = "yes"
        if 'selected_question' in sdgs[12].get('class'):
            has_sdg12 = "yes"
        if 'selected_question' in sdgs[13].get('class'):
            has_sdg13 = "yes"
        if 'selected_question' in sdgs[14].get('class'):
            has_sdg14 = "yes"
        if 'selected_question' in sdgs[15].get('class'):
            has_sdg15 = "yes"
        if 'selected_question' in sdgs[16].get('class'):
            has_sdg16 = "yes"
        if 'selected_question' in sdgs[17].get('class'):
            has_sdg17 = "yes"

          
    return (has_sdg1, has_sdg2, has_sdg3, has_sdg4, has_sdg5, has_sdg6, has_sdg7, has_sdg8, has_sdg9, has_sdg10, has_sdg11, has_sdg12,
             has_sdg13, has_sdg14, has_sdg15, has_sdg16, has_sdg17)

In [ ]:
participants = gc_full_list_soup.find_all("tr")
pdfs = {}

num_pdfs = 0
num_nonpdfs = 0
num_noreport = 0

langregex = re.compile(r'(?<=\()[^\)\(]+(?=\)$)')

print("Getting details of each report ...")
for participant in participants:
    cells = participant.find_all('td')
    company = cells[0].get_text(strip=True)
    sector = cells[1].get_text(strip=True)
    country = cells[2].get_text(strip=True)
    year = cells[3].get_text(strip=True)

    participant_entry_url = gc_base_url + cells[0].a.get('href')
    participant_profile = requests.get(participant_entry_url)
    participant_profile_soup = BeautifulSoup(participant_profile.content, 'lxml')

    (participant_sdgs_1, participant_sdgs_2, participant_sdgs_3, participant_sdgs_4, participant_sdgs_5, participant_sdgs_6, participant_sdgs_7, participant_sdgs_8,
     participant_sdgs_9, participant_sdgs_10, participant_sdgs_11, participant_sdgs_12, 
      participant_sdgs_13, participant_sdgs_14, participant_sdgs_15, participant_sdgs_16, participant_sdgs_17) = check_sdgs(participant_profile_soup)

    main_body = participant_profile_soup.find("section", class_='main-content-body')
    list_items = main_body.find_all("li")
    found_report = False
    for li in list_items:
        if li.a:
            link = li.a.get('href')
            if "/attachments/" in link:
                if ".pdf" in link:
                    link = link.split('?')[0]
                    num_pdfs += 1
                    language = langregex.search(li.get_text(strip=True))[0]
                    pdfs[link] = {"company": company, "sector" : sector, "country" : country, "year" : year, "language" : language, "sdgs1" : participant_sdgs_1, "sdgs2" : participant_sdgs_2, "sdgs3" : participant_sdgs_3, "sdgs4" : participant_sdgs_4,
                                  "sdgs5" : participant_sdgs_5, "sdgs6" : participant_sdgs_6, "sdgs7" : participant_sdgs_7, "sdgs8" : participant_sdgs_8, "sdgs9" : participant_sdgs_9, "sdgs10" : participant_sdgs_10,
                                  "sdgs11" : participant_sdgs_11, "sdgs12" : participant_sdgs_12, "sdgs13" : participant_sdgs_13, "sdgs14" : participant_sdgs_14, "sdgs15" : participant_sdgs_15,
                                  "sdgs16" : participant_sdgs_16, "sdgs17" : participant_sdgs_17,}
                    print(".", end='')
                else:
                    num_nonpdfs += 1
                found_report = True
    if not found_report:
        num_noreport += 1
print(" done.")
print("PDFs: %d, non-PDFs: %d, no-report: %d" % (num_pdfs, num_nonpdfs, num_noreport))

In [ ]:
#Saving index of reports so that it can be reused
reports_index_csv_filename = "reports_index.csv"

df_pdfs = pd.DataFrame.from_dict(pdfs, orient='index')
df_pdfs.to_csv(reports_index_csv_filename, sep='\t', encoding='utf-8')


---
## Starting point if there is already reports_index.csv file
This can be used when an index file is available (has been saved previously). Only run this cell if starting from this point, otherwise skip it. 

In [10]:
#load reports_index
reports_index_csv_filename = "reports_index.csv"

df_pdfs = pd.read_csv(reports_index_csv_filename, sep='\t', encoding='utf-8', index_col=0, dtype={'year': object})
pdfs = df_pdfs.to_dict(orient='index')
df_pdfs

,company,sector,country,year,language,sdgs1,sdgs2,sdgs3,sdgs4,sdgs5,...,sdgs8,sdgs9,sdgs10,sdgs11,sdgs12,sdgs13,sdgs14,sdgs15,sdgs16,sdgs17
//ungc-production.s3.us-west-2.amazonaws.com/attachments/cop_2021/497724/original/UNGC_APEA CoP 2021 19th May 2021 final.pdf,africapractice,Support Services,Kenya,2021,English,yes,yes,yes,yes,yes,...,yes,yes,no,yes,no,yes,no,yes,yes,yes
//ungc-production.s3.us-west-2.amazonaws.com/attachments/cop_2021/497720/original/Memoria de sostenibilidad 2020.pdf,"Adra Ingenieria y Gestion del Medio, ...",Support Services,Spain,2021,Spanish,no,no,no,no,yes,...,no,no,yes,yes,yes,yes,no,yes,no,yes
//ungc-production.s3.us-west-2.amazonaws.com/attachments/cop_2021/497721/original/Carta CEO.pdf,"Adra Ingenieria y Gestion del Medio, ...",Support Services,Spain,2021,Spanish,no,no,no,no,yes,...,no,no,yes,yes,yes,yes,no,yes,no,yes
//ungc-production.s3.us-west-2.amazonaws.com/attachments/cop_2021/497718/original/UN Global Compact Annual Report and Plan for Primeast Ltd May 2021.pdf,Primeast Ltd,Support Services,United Kingdom,2021,English,yes,yes,yes,yes,yes,...,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes
//ungc-production.s3.us-west-2.amazonaws.com/attachments/cop_2021/497716/original/Legance_COP 2021_Definitiva.pdf,Legance - Avvocati Associati,Support Services,Italy,2021,Italian,no,no,yes,yes,yes,...,yes,no,no,no,yes,yes,no,no,yes,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
//ungc-production.s3.us-west-2.amazonaws.com/attachments/9700/original/Quartzite_Ltd_COP_2010.pdf,Quartzite Ltd.,Industrial Metals & Mining,Georgia,2011,English,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,no
//ungc-production.s3.us-west-2.amazonaws.com/attachments/9697/original/110224_Arla_Foods_CSR_ENG_final.pdf,Arla Foods amba,Food Producers,Denmark,2011,English,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,no
//ungc-production.s3.us-west-2.amazonaws.com/attachments/9698/original/110223_Arla_Foods_CSR_Rapport_DK_final.pdf,Arla Foods amba,Food Producers,Denmark,2011,Danish,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,no
//ungc-production.s3.us-west-2.amazonaws.com/attachments/9699/original/110223_Arla_Foods_CSR_Rapport_SE_final.pdf,Arla Foods amba,Food Producers,Denmark,2011,Other,no,no,no,no,no,...,no,no,no,no,no,no,no,no,no,no


In [11]:
companies = {}
countries = {}
sectors = {}
years = {}
languages = {}


for pdf in pdfs.keys():
    company = pdfs[pdf]["company"]
    language = pdfs[pdf]["language"]
    year = pdfs[pdf]["year"]
    country = pdfs[pdf]["country"]
    sector = pdfs[pdf]["sector"]

    companies[company] = companies.get(company,0) + 1
    sectors[sector] = sectors.get(sector,0) + 1
    countries[country] = countries.get(country,0) + 1
    years[year] = years.get(year,0) + 1
    languages[language] = languages.get(language,0) + 1    
 

## 2. Selecting COP reports that match required criteria (up to focus_year, written in focus_language)

In [12]:
selected_companies = {}
selected_sectors = {}
selected_countries = {}
selected_years = {}
selected_countries_years = {}

selected_pdfs = {}

for pdf in pdfs.keys():
    company = pdfs[pdf]["company"]
    language = pdfs[pdf]["language"]
    year = pdfs[pdf]["year"]
    country = pdfs[pdf]["country"]
    sector = pdfs[pdf]["sector"]

#    if language == language_ref[focus_language]['name'] and int(year) <= int(focus_year):
    if language == language_ref[focus_language]['name'] and int(year) == int(focus_year): #only for the focus_year
        selected_pdfs[pdf] = pdfs[pdf]
        
        selected_companies[company] = selected_companies.get(company,0) + 1
        selected_sectors[sector] = selected_sectors.get(sector,0) + 1
        selected_countries[country] = selected_countries.get(country,0) + 1
        selected_years[year] = selected_years.get(year,0) + 1
        if country in selected_countries_years.keys():
            selected_countries_years[country][year] = selected_countries_years[country].get(year,0) + 1
        else:
            selected_countries_years[country] = {year : 1}

In [13]:
print("There are %d reports up to %s written in %s" % (len(selected_pdfs.keys()), focus_year, language_ref[focus_language]['name']))

There are 1627 reports up to 2021 written in English


## 3. Downloading PDF file for each COP report that matches required criteria
At this time we've only considered reports written in the focus language and submitted up to end of the focus year.

A folder should be specified as the location where PDFs will be downloaded to ('pdfs_folder' variable below).

If this process has been run before and some files are already available in the specified folder, they won't be downloaded again.

In [16]:
pdfs_folder = "../data/pdf/"

In [ ]:
filenameregex = re.compile(r'(?<=/)[^$/]+(?=$)')

try:
    os.stat(pdfs_folder)
except:
    os.mkdir(pdfs_folder) 

for pdf in selected_pdfs.keys():
    filename = pdfs_folder + filenameregex.search(pdf)[0]

    if not os.path.isfile(filename):
        print("Saving %s" % (filename))
        file = requests.get('https:'+ pdf, stream=True)
        try:
            with open(filename, 'wb') as out_file: #file handler needed
                shutil.copyfileobj(file.raw, out_file) #file name closed needed
            del file
        except:
            print("Could not save %s" % (filename))
            continue
        
    else:
        print("Skipping %s, PDF already available in folder" % (filename))

Skipping ../data/pdf/UNGC_APEA CoP 2021 19th May 2021 final.pdf, PDF already available in folder
Skipping ../data/pdf/UN Global Compact Annual Report and Plan for Primeast Ltd  May 2021.pdf, PDF already available in folder
Skipping ../data/pdf/STRAX_2020_Sustainability report_COP.pdf, PDF already available in folder
Skipping ../data/pdf/2021 Communication on Progress %28002%29.pdf, PDF already available in folder
Saving ../data/pdf/Sustainability and Impact Report 2020.pdf
Saving ../data/pdf/LID_COP_2021.pdf
Saving ../data/pdf/CSR Report  Grant Thornton 2021.pdf
Saving ../data/pdf/UNGC COP 2021  Mekoprint.pdf
Saving ../data/pdf/EVRAZ_SR_2020_eng_final.pdf
Saving ../data/pdf/St1_Integrated_report__2020_110521.pdf
Saving ../data/pdf/Royal Auping CSR Report May 2021.pdf
Saving ../data/pdf/Rapport_activit_AnGao_20202021.pdf
Saving ../data/pdf/GLOBAL COMPACT 2020 Final.pdf
Saving ../data/pdf/Global Compact Commitment Letter 2020 Final.pdf
Saving ../data/pdf/Tenco COP 2021.pdf
Saving ../data

Saving ../data/pdf/UNIQA_Insurance_Group_AG_UNGC_COP_2020.pdf
Saving ../data/pdf/afrasiasustainabilityreport2020.pdf
Saving ../data/pdf/Nokian_Tyres_corporate_sustainability_report_2020.pdf
Saving ../data/pdf/CorporateResponsibilityReportHL2020.pdf
Saving ../data/pdf/Nordex Food CSRrapport 2020.pdf
Saving ../data/pdf/2020_tx_sustainability_report_en.pdf
Saving ../data/pdf/SMICIntegratedReport2020.pdf
Saving ../data/pdf/REPORTE CMPC 2020 ENG_5 mayo %281%29.pdf
Saving ../data/pdf/SSL_SustainabilityReport_2020.pdf
Saving ../data/pdf/Albany International Corp.  UN CoP 05052021.pdf
Saving ../data/pdf/Rosendahl_COP_rapport_2020_280421FinalOpslag.pdf
Saving ../data/pdf/NRS Relief  COP 20192020.pdf
Saving ../data/pdf/Building a Better Coffee World  1920 Sustainability Report.pdf
Saving ../data/pdf/UNGC_COP_Swarovski_2020_FINAL.pdf
Saving ../data/pdf/Sustainability_Report_CityMail_Sweden_2020.pdf
Saving ../data/pdf/COP ODDO BHF 2021.pdf
Saving ../data/pdf/OSM_Sustainability_Report_2020_ FINAL.p

Saving ../data/pdf/GPT Sustainability Report 2020.pdf
Saving ../data/pdf/JORDENENCOP2021.pdf
Saving ../data/pdf/907PLCY000019.pdf
Skipping ../data/pdf/2020 Communication on Progress.pdf, PDF already available in folder
Saving ../data/pdf/HMY_Corporate Sustainability Report COP 2021.pdf
Saving ../data/pdf/2021 Global Responsibility Report Final.pdf
Saving ../data/pdf/Informe de Progreso 2020  Grupo SAN f.pdf
Saving ../data/pdf/2020 COP FINAL UPLOADED.pdf
Saving ../data/pdf/RetturUN CeoCoverLette2604201final.pdf
Saving ../data/pdf/RetturCOP26042021 Final.pdf
Saving ../data/pdf/Daher_2021_COP_CEO Statement of support_202104.pdf
Saving ../data/pdf/UNGC_Communication_on_Progress_2020.pdf
Saving ../data/pdf/FY2020 GC Annual Report.pdf
Saving ../data/pdf/2019SustainabilityReport.pdf
Saving ../data/pdf/ARVAL_COP 2020_April 23rd 2021.pdf
Saving ../data/pdf/Arval CEO COP Letter.pdf
Saving ../data/pdf/2020_HEINE_Optotechnik_COP.pdf
Saving ../data/pdf/csr_report2019_en.pdf
Saving ../data/pdf/UN Gl

Skipping ../data/pdf/IWBI FINAL UN Global Compact Report 201920 w_letter.pdf, PDF already available in folder
Saving ../data/pdf/COP2020_EN_25march2021.pdf
Saving ../data/pdf/Communication_on_engagement_2021.pdf
Saving ../data/pdf/Linxens Communication on Progress 2020.pdf
Saving ../data/pdf/Vossloh UNGC Communication on Progress Report_2020.pdf
Saving ../data/pdf/COP reporting.pdf
Saving ../data/pdf/Communication on Progress WorleChemie 2021 english.pdf
Saving ../data/pdf/Sustainability report_20162018.pdf
Skipping ../data/pdf/Communication on Progress WorleChemie 2021 english.pdf, PDF already available in folder
Skipping ../data/pdf/Sustainability report_20162018.pdf, PDF already available in folder
Saving ../data/pdf/Communication on Progress 2018 to 2019.pdf
Saving ../data/pdf/COP_2020_2021.pdf
Saving ../data/pdf/Sustainovate Progress Report 2020.pdf
Saving ../data/pdf/Statement of Continuous Support and COP 2021 and Encls..pdf
Saving ../data/pdf/BWZ__UN_Global_Compact_2020.pdf
Sav

Skipping ../data/pdf/Communication on Progress 2020.pdf, PDF already available in folder
Saving ../data/pdf/2019 CSR Report.pdf
Saving ../data/pdf/CSR Report 2020_final.pdf
Saving ../data/pdf/KFIVEKINNARPS.pdf
Saving ../data/pdf/FM_srep2020E_all.pdf
Saving ../data/pdf/shimadzu_integrated_report_2020a.pdf
Saving ../data/pdf/Carlson COP 2021.pdf
Saving ../data/pdf/Aviancas Communication on progress 2020.pdf
Saving ../data/pdf/Sutherland_UN_COP Letter 2021.pdf
Skipping ../data/pdf/COP2021.pdf, PDF already available in folder
Saving ../data/pdf/Dukat_Corporate Social Responsibility Report 2020_ENG.pdf
Saving ../data/pdf/FoodXervices COP Revised.pdf
Skipping ../data/pdf/sustainability_report.pdf, PDF already available in folder
Saving ../data/pdf/COP ETOJ 2020.pdf
Saving ../data/pdf/OTB Consult  Communication on Progress.pdf
Saving ../data/pdf/Marvell UNGC COP 2021 FINAL .pdf
Saving ../data/pdf/2020 Sarnova UNGC COP.pdf
Skipping ../data/pdf/Arval CEO COP Letter.pdf, PDF already available in

Saving ../data/pdf/marel2020esgreport.pdf
Saving ../data/pdf/Securitas_AR2020_eng.pdf
Saving ../data/pdf/COP 2020_CFMoller_For upload.pdf
Saving ../data/pdf/GreenMobility ESG  Sustainability Report 2020.pdf
Saving ../data/pdf/UN COP 2021 vFinal.pdf
Saving ../data/pdf/Communication on progress 2020 final.pdf
Saving ../data/pdf/Semperit_NFR2020_EN_SP_210317_incl. CEO Statement UNGC.pdf
Saving ../data/pdf/Autoliv_Sustainability_Report_2020.pdf
Saving ../data/pdf/2020 OMON Group Inc COP with attachment.pdf
Saving ../data/pdf/Hexagon Annual Report 2020.pdf
Saving ../data/pdf/COP_VKR_HOLDING_2020.pdf
Saving ../data/pdf/CP ALL SD 2020 Eng Final_2%28COP%29.pdf
Saving ../data/pdf/casa_urd2020_uk_final.pdf
Saving ../data/pdf/ISSEE  UN  Global Compact Communication on Progress  Mar 2020  Mar 2021.pdf
Saving ../data/pdf/AuOne COP  2020.pdf
Saving ../data/pdf/Artmatrix COP 2020.pdf
Skipping ../data/pdf/Artmatrix COP 2020.pdf, PDF already available in folder
Saving ../data/pdf/SKGAS_ENG.pdf
Saving .

Saving ../data/pdf/NBG_SUSTAINABILITY REPORT 2019.pdf
Saving ../data/pdf/TUIGroupSustainabilityReport2019_Final.pdf72ad05bff1496edcbe875bb827d93e41.pdf
Saving ../data/pdf/cloettacloettasannualreport2020availableonthewebsite210315.pdf
Saving ../data/pdf/COP April2021.pdf
Saving ../data/pdf/CSR_Report_2021.pdf
Saving ../data/pdf/KMD 2021 Annual COP for UN Global Compact Final.pdf
Saving ../data/pdf/COP_20210331_en.pdf
Saving ../data/pdf/GS Yuasa CSR Report 2020 %28English%29.pdf
Saving ../data/pdf/2020_Financial_and_ESG_Report.pdf
Saving ../data/pdf/ODSReport_eng.pdf
Saving ../data/pdf/fibra_danhos_SR_2019 %281%29.pdf
Saving ../data/pdf/Lantmnnen Annual and Sustainability Report 2020.pdf
Saving ../data/pdf/Amasten Global Compact Statement 2020.pdf
Saving ../data/pdf/sustainabilityp7s1ar20.pdf
Saving ../data/pdf/AdEx Partners UNGC Report 2021.pdf
Saving ../data/pdf/tesa SE  Communication on Progress 2020.pdf
Saving ../data/pdf/Integrated Report 2019 ENG.pdf
Saving ../data/pdf/Rapport_CSR 

Saving ../data/pdf/AR20192020.pdf
Saving ../data/pdf/INFORME COP 2019  2020 Oriente Seguros.pdf
Saving ../data/pdf/Legal  General Sustainability Report.pdf
Saving ../data/pdf/2020_COP_Ingka_Group.pdf
Saving ../data/pdf/IngkaGroupAnnualSummarySustainabilityReportFY20.pdf
Skipping ../data/pdf/COP 2020.pdf, PDF already available in folder
Saving ../data/pdf/CSR_2020.pdf
Saving ../data/pdf/The_LEGO_Group_Sustainability_progress_2020_FINAL.pdf
Saving ../data/pdf/CoP_2020.pdf
Skipping ../data/pdf/CoP_2020.pdf, PDF already available in folder
Saving ../data/pdf/Communication on progress %28 YTCI Global Compact 20202021 UNGC %29 %281%29.pdf
Saving ../data/pdf/CTICCAnnualReport2020.pdf
Saving ../data/pdf/Neste Annual Report 2020.pdf
Saving ../data/pdf/Mercan COP 2021.pdf
Saving ../data/pdf/lassilatikanojaannualreport2020annualreview.pdf
Saving ../data/pdf/Matex_COP_2020.pdf
Saving ../data/pdf/UN Global Compact  Communication on Progress March 2021.pdf
Saving ../data/pdf/Debido al contexto  de p